# Homework 1: Locally Sensitive Hashing
Locality Sensitive Hashing (LSH) is a technique used in computer science to solve the approximate or exact Near Neighbor Search in high-dimensional spaces. It is used to find similar items in a large dataset by hashing input items so that similar items map to the same "buckets" with high probability. LSH is commonly used in recommendation systems, image and audio recognition, and data mining.

In this particular notebook we will implement a simplified version of the LSH algorithm for to compare texts and find how similar are. We will implement 4 classes which will help us to compute how similar 2 texts are. Those classes are: Shingling, CompareSets, MinHasing and CompareSignatures.

## Dataset
As a dataset, we have used the following texts:
 - Lorem Ipsum with 5 paragraphs (https://www.lipsum.com/)\[1.txt\]
 - Lorem Ipsum with 7 paragraphs (https://www.lipsum.com/)\[2.txt\]
 - Quijote de la Mancha by Miguel de Cervantes (https://www.gutenberg.org/cache/epub/60884/pg60884.txt)\[3.txt\]
 - The Adventures of Sherlock Holmes by Arthur Conan Doyle (https://www.gutenberg.org/cache/epub/1661/pg1661.txt)\[4.txt\]
 - The picture of Dorian Gray by Oscar Wilde (https://www.gutenberg.org/cache/epub/174/pg174.txt)\[5.txt\]
 - Beyond good and evil by Friedrich Wilheim Nietzsche (https://www.gutenberg.org/cache/epub/4363/pg4363.txt)\[6.txt\]

This dataset has been selected like this, so it has two text which should be fairly similar (1 & 2), a text which should be fairly different (3) and three texts which should have something in common even though they may be different (4, 5 & 6).

In this notebook we have implemented 4 classes which we will help us to compute the similarity of 2 texts. Those classes are: Shingling, CompareSets, MinHashing and CompareSignatures.

In [10]:
import os

# define the path to the dataset folder
dataset_path: str = "dataset"

# create an empty list to store the texts
texts: list[str] = []

# loop through the files in the dataset folder
for filename in os.listdir(dataset_path):
    # check if the file is a text file
    if filename.endswith(".txt"):
        # open the file and read its contents
        with open(os.path.join(dataset_path, filename), "r") as f:
            text = f.read()
            texts.append(text)

# print the texts
for i, text in enumerate(texts):
    print(f"Text {i+1}:")
    print(text)
    print("-"*132)


Text 1:
HAPTER I.


The studio was filled with the rich odour of roses, and when the light
summer wind stirred amidst the trees of the garden, there came through
the open door the heavy scent of the lilac, or the more delicate
perfume of the pink-flowering thorn.

From the corner of the divan of Persian saddle-bags on which he was
lying, smoking, as was his custom, innumerable cigarettes, Lord Henry
Wotton could just catch the gleam of the honey-sweet and honey-coloured
blossoms of a laburnum, whose tremulous branches seemed hardly able to
bear the burden of a beauty so flamelike as theirs; and now and then
the fantastic shadows of birds in flight flitted across the long
tussore-silk curtains that were stretched in front of the huge window,
producing a kind of momentary Japanese effect, and making him think of
those pallid, jade-faced painters of Tokyo who, through the medium of
an art that is necessarily immobile, seek to convey the sense of
swiftness and motion. The sullen murmur of 

## Shingling
Shingling is a technique used in text analysis to represent a document as a set of overlapping subsequences of fixed length k, called k-shingles. To compute shingles, we slide a window of size k over the document and extract the k-length substrings that fall within the window. We then store these substrings as a set, which represents the shingles of the document.

In [2]:
class Shingling():
    
    def __init__(self, k: int):
        self.k: int = k
        
    def get_shingles(self, document: str) -> set:
        """
        This method constructs k-shingles of a given length k from a given document.
        """
        shingles: set = set()
        for i in range(len(document) - self.k + 1):
            shingle: str = document[i:i+self.k]
            shingles.add(shingle)
        return shingles
    
    def get_hashed_shingles(self, document: str):
        """
        This method computes a hash value for each unique shingle and represents the document in the form of an ordered set of its hashed k-shingles.
        """
        shingles: set = self.get_shingles(document)
        hashed_shingles: list[int] = [hash(shingle) for shingle in shingles]
        hashed_shingles.sort()
        return hashed_shingles

As an example to show what is shingling, we will use the following text:
> "The quick brown fox jumps over the lazy dog"

In [9]:
text = "The quick brown fox jumps over the lazy dog"
shingling: Shingling = Shingling(k=5)
shingles: list[str] = shingling.get_shingles(text)
print(shingles)
shingles_hash: list[int] = shingling.get_hashed_shingles(text)
print("-"*132)
print(shingles_hash)



{'s ove', 'ps ov', ' over', ' quic', 'er th', 'uick ', 'x jum', 'the l', ' lazy', 'jumps', 'ck br', 'azy d', ' jump', 'own f', 'y dog', 'zy do', ' fox ', 'ick b', 'ox ju', 'umps ', 'he la', 'brown', 'he qu', 'n fox', ' the ', 'fox j', 'quick', 'over ', 'e laz', 'lazy ', 'mps o', 'wn fo', 'e qui', 'k bro', 'rown ', 'ver t', 'The q', ' brow', 'r the'}
------------------------------------------------------------------------------------------------------------------------------------
[-9064995236138712210, -9005714578446310487, -8983754845269442767, -8927896972988628871, -8630367961634716046, -8258025870622758380, -8120242463804657335, -6216759897822607635, -5858672634050728466, -5621998960879170018, -5595517215378991960, -5021308199409380416, -4385432165039772672, -3728321350543117239, -3566298837497330793, -2954428360512192576, -2686088662228484159, -1599106652430495275, -1464095131349026324, -1458535426855079801, -1116109200230187167, -870352256373828993, -112010859600308847, -691078162

Now we do it with our text. We use a k=9 as we are analyzing text from books instead of emails. In the following lines we are doing exactly the same as we showed in the example above.

In [11]:
shingling: Shingling = Shingling(k=9)

hashed_shingles_list: list[int] = []

for text in texts:
    hashed_shingles: int = shingling.get_hashed_shingles(text)
    hashed_shingles_list.append(hashed_shingles)
    
for i, hashed_shingles in enumerate(hashed_shingles_list):
    print(f"Hashed shingles for Text {i+1}:")
    print(hashed_shingles)
    print("-"*132)


Hashed shingles for Text 1:
[-9219181323273320679, -9216823089655708748, -9216729819939597133, -9206084689099071493, -9204010052478935175, -9202810784080110787, -9201844094832180714, -9193039105951235071, -9192729103466699049, -9188155234046251688, -9184168112132462594, -9180249918495552565, -9175323189330268000, -9173376824755128192, -9170788716443793835, -9161943151853898838, -9156559737966868522, -9155672173439712190, -9149883967593840406, -9141480378783834217, -9123022513216886601, -9122317986611332552, -9121945463584807855, -9115462522759781870, -9106801755252932238, -9100739440169014255, -9095948251672969577, -9094077837340457311, -9069372105604589312, -9068068689053633941, -9066177495498358596, -9063954983227836658, -9060511420167824431, -9058073933696635945, -9055666857993212281, -9054666375179445574, -9045154612164484455, -9030294010354745626, -9029477143162461636, -9029246188014751532, -9025695292853345504, -9004994491779768356, -9004955051170904359, -8991746809522671753, -89

## CompareSets


## MinHashing

## CompareSignatures

Alina 1:05